# Physical Fitness Test Data Preprocessing
- Dataset Download (2007-2013): https://www.cde.ca.gov/ta/tg/pf/pftresearch.asp
- Kenny Chen
- Group 5, ECE 143 Spring 2018


#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from io import StringIO
import zipfile
import re

#### Parse Raw Zip Files

In [2]:
# read zip files
data_zip = glob('./pft_data/*.zip')

# extract contents of each zip file into folder with same name
for zf in data_zip:
  with zipfile.ZipFile(zf,'r') as z:
    # define folder name
    folder_name = zf[0:-4]
    
    # extract contents
    z.extractall(folder_name)

#### Parse Data Folders

In [3]:
# get list of all folders
data_folders = glob("./pft_data/*/")

# sort the folders by increasing year
list.sort(data_folders)

#### Parse Fitness Data into Dictionaries

In [ ]:
# create dictionary to contain all data
data = dict()

for folder_num, folder in enumerate(data_folders):
  # create dictionary of years within dictionary for each folder
  key = folder[-3:-1]
  data[key] = dict()
  
  print('==== PARSING FITNESS DATA FROM 20' + key + ' (' + str(folder_num+1) + ' of ' + str(len(data_folders)) + ') ====')
  
  # get list of text files in folder
  txt_files = glob(folder+'*.txt')
  
  # read txt files into pandas dataframe and store
  for filename in txt_files:
    # get number of header columns
    num_cols = len(list(pd.read_csv(filename, nrows=0)))
    
    # get deliminiter depending on year
    if int(key) < 12:
      dlim = '|'
    elif int(key) >= 12:
      dlim = '\t'
    
    # change from comma-delimited to | delimited to account for extra commas in name in last column, but only for datasets before 2012
    temp = StringIO()
    with open(filename, 'r', encoding='cp1250') as f:
      for line in f:
        if int(key) < 12:
          new_line = re.sub(r',', '|', line.rstrip(), count=num_cols-1)
        elif int(key) >= 12:
          new_line = line
        print (new_line, file=temp)
        
    temp.seek(0)
    temp_df = pd.read_csv(temp, sep=dlim, low_memory=False)
    temp.seek(0)
    
    # parse county codes
    if 'entities' in filename.lower():
      # read only county code and county name columns
      if temp_df.columns.isin(['ccode']).any():
          df = pd.read_csv(temp, sep=dlim, usecols=['ccode','County'], low_memory=False)
      elif temp_df.columns.isin(['Ccode']).any():
          df = pd.read_csv(temp, sep=dlim, usecols=['Ccode','County'], low_memory=False)
        
      # remove duplicate rows
      df.drop_duplicates(inplace=True)
      
      # store into dictionary
      data[key]['county_codes'] = df
    
    # parse actual fitness data
    if 'research' in filename.lower() or 'physfit' in filename.lower():
      # old and new datasets have different column names
      cols_old = ['Ccode','Gr9PctIn']
      cols_new = ['CO','Perc9a']
      
      if sum(temp_df.columns.isin(cols_old)) == 2:
        # read only county code and county name columns from county-level aggregations
        try:
          temp.seek(0)
          df = pd.read_csv(temp, sep=dlim, usecols=['Level_Number','Ccode','Gr9PctIn'], low_memory=False)
          temp.seek(0)
          lvl = 'Level_Number'
        except:
          temp.seek(0)
          df = pd.read_csv(temp, sep=dlim, usecols=['Level','Ccode','Gr9PctIn'], low_memory=False)
          temp.seek(0)
          lvl = 'Level'
          
        # replace asterisks with NaN
        df['Gr9PctIn'].replace('*', np.nan, inplace=True)
        df['Gr9PctIn'].replace('**', np.nan, inplace=True)
        
        # convert to numeric
        df['Gr9PctIn'] = pd.to_numeric(df['Gr9PctIn'])
        
        # take only rows with finite values
        df = df[np.isfinite(df['Gr9PctIn'])]
        
        # take only county aggregation values
        df = df.loc[df[lvl] == 3]
        df = df.drop([lvl], axis=1)
        
        # take the mean of the countys
        df = df.groupby(['Ccode']).mean()
        
        # store into dictionary
        data[key]['county_data'] = df
      
      elif sum(temp_df.columns.isin(cols_new)) == 2:
        # read only county code and county name columns from county-level aggregations
        try:
          temp.seek(0)
          df = pd.read_csv(temp, sep=dlim, usecols=['Level_Number','CO','Perc9a'], low_memory=False)
          temp.seek(0)
          lvl = 'Level_Number'
        except:
          temp.seek(0)
          df = pd.read_csv(temp, sep=dlim, usecols=['Level','CO','Perc9a'], low_memory=False)
          temp.seek(0)
          lvl = 'Level'
          
        # replace asterisks with NaN
        df['Perc9a'].replace('*', np.nan, inplace=True)
        df['Perc9a'].replace('**', np.nan, inplace=True)
        
        # convert to numeric
        df['Perc9a'] = pd.to_numeric(df['Perc9a'])
        
        # take only rows with finite values
        df = df[np.isfinite(df['Perc9a'])]
        
        # take only county aggregation values
        df = df.loc[df[lvl] == 3]
        df = df.drop([lvl], axis=1)
        
        # take the mean of the countys
        df = df.groupby(['CO']).mean()
        
        # store into dictionary
        data[key]['county_data'] = df

==== PARSING FITNESS DATA FROM 2007 (1 of 7) ====
==== PARSING FITNESS DATA FROM 2008 (2 of 7) ====
==== PARSING FITNESS DATA FROM 2009 (3 of 7) ====
==== PARSING FITNESS DATA FROM 2010 (4 of 7) ====
==== PARSING FITNESS DATA FROM 2011 (5 of 7) ====


#### Parse Fitness Data into Single DataFrame

In [ ]:
# extract County names for DataFrame columns
cols = data['13']['county_codes']['County'].values.tolist()[0:58]

# create index array for DataFrame rows
idx= []
for folder in data_folders:
  year = '20' + folder[-3:-1]
  idx.append(year)

# initialize data matrix of extracted fitness data
data_mat = np.zeros(shape=[len(data_folders),len(cols)])

# fill in extracted fitness data
for folder_num, folder in enumerate(data_folders):
  # extract key
  key = folder[-3:-1]
  
  # insert fitness data for relevant counties
  for i, row in data[key]['county_data'].iterrows():
    if int(row.name) >= 1 and int(row.name) <= 58:
      data_mat[folder_num, i-1] = row.values

# create DataFrame
pft_df = pd.DataFrame(data_mat, index=idx, columns=cols)

#### Save to Pickle File

In [ ]:
pft_df.to_pickle('./physical_fitness_data.pkl')